In [263]:
import os
import re
import codecs
import gensim
import pymorphy2
#import codecs

In [264]:
import zipfile
with zipfile.ZipFile('180.zip', 'r') as archive:
    stream = archive.open('model.bin')
    sem_model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [265]:
cotags = {'ADJF':'ADJ', # pymorphy2: word2vec 
'ADJS' : 'ADJ', 
'ADVB' : 'ADV', 
'COMP' : 'ADV', 
'GRND' : 'VERB', 
'INFN' : 'VERB', 
'NOUN' : 'NOUN', 
'PRED' : 'ADV', 
'PRTF' : 'ADJ', 
'PRTS' : 'VERB', 
'VERB' : 'VERB'}

In [266]:
def cleantext(sometext):
    sometext = re.sub ('\n|\ni','', sometext) 
    sometext = re.sub ('(.)([.,!?;:])','\\1 \\2', sometext)
    return (sometext)

In [267]:
def tokenize (string):
    return string.split(' ')

In [268]:
def get_gram_tag (tag):
    ud_POS = 'NONE'
    this_pos = tag.POS
    if this_pos in cotags:
        ud_POS = cotags [this_pos]
    return ud_POS

In [269]:
def choose_simword (simwords, original_pos):
    for simword in simwords:
        word_tag = simword[0]
        tag = word_tag.split ('_')[1]
        if tag == original_pos:
            return (word_tag)
    return simwords [0][0]
    

In [270]:
def get_most_similar (word_and_tag, sem_model):
    word, tag = word_and_tag
    ud_pos_tag = get_gram_tag (tag)
    simword = word
    print (word)
    print (tag)
    word_for_vec = word +'_' + ud_pos_tag
    
    if word_for_vec in model:
        print (word_for_vec)
        simwords = model.most_similar(word_for_vec, topn=10)
        simword = choose_simword (simwords, ud_pos_tag) #[0][0]
        print ('got synonym {} {}'.format(word, simword))
    #simwords = model.most_similar(positive=[word], topn=10):
    return (simword)

In [271]:
def replace_words_with_vectsyns (wordlist):
    #open_binary_model = open ('model.bin')
    #my_model = gensim.models.KeyedVectors.load_word2vec_format(open_binary_model, binary=True)
    newlist = []
    for parse in wordlist:
        simword = get_most_similar (parse, sem_model)
        newlist.append (simword)
    return (newlist)

In [272]:
def lemmatize (wordlist):
    morph = pymorphy2.MorphAnalyzer()
    lemmatized = []
    for word in wordlist:
        parse = morph.parse(word)[0]
        lemma = parse.normal_form
        tag = parse.tag
        lemmatized.append ((lemma, tag))
    return (lemmatized)

In [273]:
def vectorize_text(inputfile, outputfile):
    text = codecs.open (inputfile, 'r', 'utf-8')
    text_as_string = text.read()
    text_as_string = cleantext (text_as_string)
    tokenized_text = tokenize (text_as_string)
    lemmas_and_grams = lemmatize (tokenized_text)
    test_piece = lemmas_and_grams[:30]
    #print (test_piece)
    vectorized = replace_words_with_vectsyns (test_piece)
    print (vectorized)
    

In [274]:
inputfile = 'bk.txt' #'Dostoevsky_PrestuplenieINakazanie.txt'
outputfile = 'vectorized_' + inputfile
vectorize_text (inputfile, outputfile)

в
PREP
один
ADJF,Apro femn,sing,gent
один_ADJ
got synonym один одинаковый_ADJ
из
PREP
отдалённый
ADJF,Qual plur,gent
наш
ADJF,Apro plur,gent
наш_ADJ
got synonym наш град_PROPN
губерния
NOUN,inan,femn plur,gent
губерния_NOUN
got synonym губерния уезд_NOUN
находиться
VERB,impf,intr neut,sing,past,indc
находиться_VERB
got synonym находиться расположенный_VERB
имение
NOUN,inan,neut sing,nomn
имение_NOUN
got synonym имение именье_NOUN
иван
NOUN,anim,masc,Name sing,gent
иван_NOUN
got synonym иван травка_NOUN
пётр
NOUN,anim,masc,Patr sing,gent
берестов
NOUN,anim,masc,Sgtm,Surn sing,gent
.
PNCT
в
PREP
молодость
NOUN,inan,femn sing,loct
молодость_NOUN
got synonym молодость юность_NOUN
свой
ADJF,Apro,Anph femn,sing,gent
свой_ADJ
got synonym свой твой_ADJ
служилый
ADJS masc,sing
служилый_ADJ


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


got synonym служилый тяглый_ADJ
он
NPRO,masc,3per,Anph sing,nomn
в
PREP
гвардия
NOUN,inan,femn sing,gent
гвардия_NOUN
got synonym гвардия гвардеец_NOUN
,
PNCT
выйти
VERB,perf,intr masc,sing,past,indc
выйти_VERB
got synonym выйти выходить_VERB
в
PREP
отставка
NOUN,inan,femn sing,accs
отставка_NOUN
got synonym отставка увольнение_NOUN
в
PREP
начало
NOUN,inan,neut sing,loct
начало_NOUN
got synonym начало начинать_NOUN
1797
NUMB,intg
год
NOUN,inan,masc sing,gent
год_NOUN
got synonym год месяц_NOUN
,
PNCT
уехать
VERB,perf,intr masc,sing,past,indc
уехать_VERB
got synonym уехать уезжать_VERB
в
PREP
['в', 'одинаковый_ADJ', 'из', 'отдалённый', 'град_PROPN', 'уезд_NOUN', 'расположенный_VERB', 'именье_NOUN', 'травка_NOUN', 'пётр', 'берестов', '.', 'в', 'юность_NOUN', 'твой_ADJ', 'тяглый_ADJ', 'он', 'в', 'гвардеец_NOUN', ',', 'выходить_VERB', 'в', 'увольнение_NOUN', 'в', 'начинать_NOUN', '1797', 'месяц_NOUN', ',', 'уезжать_VERB', 'в']


In [225]:
import zipfile
with zipfile.ZipFile('180.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)
simwords = model.most_similar('конь_NOUN', topn=10)
print (simwords[0][0])

лошадь_NOUN


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [285]:
morph = pymorphy2.MorphAnalyzer()
parse = morph.parse ('были')
#parse = morph.parse ('Иван')
tag = parse[0].tag
lemma = parse[0].normal_form
test = [(lemma,tag)]
print (test)

[('быть', OpencorporaTag('VERB,impf,intr plur,past,indc'))]


In [281]:
#open_binary_model = open ('model.bin')
#my_model = gensim.models.KeyedVectors.load_word2vec_format(open_binary_model, binary=True)
simwords = model.most_similar(positive=['конь_NOUN'], topn=10)
simwords 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('лошадь_NOUN', 0.7664846777915955),
 ('жеребец_NOUN', 0.7329774498939514),
 ('седло_NOUN', 0.723147988319397),
 ('аргамак_NOUN', 0.7189077138900757),
 ('кобылица_NOUN', 0.7155561447143555),
 ('иноходец_NOUN', 0.7025481462478638),
 ('скакун_NOUN', 0.6938524842262268),
 ('кобыла_NOUN', 0.6869013905525208),
 ('конь_PROPN', 0.686468780040741),
 ('конь_VERB', 0.652535617351532)]